#### PDF RAG SEARCH WITH OLLAMA AND CREWAI

Steps to install ollama
1. url->https://ollama.com/download download the ollama setup from here and execute the installation
2. ollama pull phi3:3.8b -> pulling the open ai model, which has been trained on 3.8 parameters
3. check if ollama server is up and running by visiting this url-> http://localhost:11434/

- Input used for this model is a research paper on deepseek-> "DeepSeek-VL: Towards Real-World Vision-Language
Understanding" 
- url of pdf -> https://arxiv.org/abs/2403.05525
- github link- https://github.com/ollama/ollama
- more on installation of ollama https://www.youtube.com/watch?v=UtSSMs6ObqY

##### STEP 1: Import the libraries

In [1]:
import os
from crewai import Agent,Task,Crew
from langchain_openai import ChatOpenAI
from langchain_community.llms import Ollama

##### STEP 2: Giving a dummy api key as the OpenAI chat has a placeholder for this

In [2]:
os.environ["OPENAI_API_KEY"]="sk-proj-1234"

##### STEP3: Let's define the ollama LLM

In [3]:
llm = ChatOpenAI(
    model='ollama/phi3:latest',
    base_url="http://localhost:11434/v1",
    api_key="sk-proj-1234"
)

##### STEP 4: Let's define the tool

In [4]:
from embedchain import App
from embedchain.embedder.ollama import OllamaEmbedder
from crewai_tools.tools.pdf_search_tool.pdf_search_tool import PDFSearchTool

Now since we are using a local AI model ollama, crew uses embedchain to perform RAG searches, so we to use ollama 
- for embedding
- for query 

In [5]:
import os
from embedchain import App
from embedchain.models.data_type import DataType  # Import the correct data type

# Set the Ollama host (if needed)
os.environ["OLLAMA_HOST"] = "http://127.0.0.1:11434"

# Load embedding model configuration from config.yaml file
app = App.from_config(config_path="config.yaml")

# Provide the correct PDF file path
pdf_path = r"D:\Courses\Crew AI youtube crash course\deepseek.pdf"  # Ensure the file exists

# Use the correct enum for data_type
app.add(pdf_path, data_type=DataType.PDF_FILE)

print("PDF added successfully!")


c:\Users\Varsha Kannan\AppData\Local\Programs\Python\Python312\Lib\site-packages\embedchain\embedder\ollama.py:27: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=self.config.model, base_url=config.base_url)
Inserting batches in chromadb:   0%|          | 0/2 [00:00<?, ?it/s]2025-01-31 17:53:28,248 - 22308 - local_persistent_hnsw.py-local_persistent_hnsw:339 - WARNING: Add of existing embedding ID: default-app-id--570d4af502dda016548c1c6cf5eb3550c4a9e59c373251008761ed7e3a790beb
2025-01-31 17:53:28,248 - 22308 - local_persistent_hnsw.py-local_persistent_hnsw:339 - WARNING: Add of existing embedding ID: default-app-id--570d4af502dda016548c1c6cf

PDF added successfully!


In [6]:
# Initialize the PDF RAG Search Tool with Ollama
pdf_rag_tool = PDFSearchTool(app=app)

##### STEP 5: Let's define the agents

In [7]:
from crewai import Agent, Task, Crew, Process

researcher=Agent(
    role="Information Agent",
    goal="Go through the query asked on the topic {topic}, and retrieve all the relevent content about the topic and write it down",
    verbose=True,
    allow_delegation=False,
    memory=True,
    backstory="""
        You are an expert in going through pdfs and understanding the content and writing a short summary about it.
    """,
    llm=llm
)

writer=Agent(
    role="Content writer",
    goal="Go through the content {topic} provided by the researcher agent and summarize and give the answer in consice and meaningful format",
    verbose=True,
    allow_delegation=False,
    memory=True,
    backstory="""You are exceptional in explaining complex topics in simple words in the form of bullet points""",
    llm=llm,
    
)

##### STEP 6: Define the tasks

In [8]:
research_task = Task(
    description="Search and collect information on {topic}.",
    expected_output="Colect all the information about {topic} and write in the form 1 paragraph.",
    agent=researcher
)

write_task=Task(
    description="Bullet points on {topic}.",
    expected_output="Write short consice bullet points of main points one below the other, in less than 30 words. ",
    agent=writer,
    async_execution=False
)

In [10]:
crew = Crew(
    agents=[researcher,writer],
    tasks=[research_task,write_task],
    verbose=True,
    tools=[pdf_rag_tool],
    process=Process.sequential
)

In [11]:
result = crew.kickoff(inputs={'topic':'Explain the performance of deepseek.'})

# Agent: Information Agent
## Task: Search and collect information on Explain the performance of deepseek..


# Agent: Information Agent
## Final Answer: 
DeepSeek's performance has demonstrated remarkable efficiency in processing large datasets within short timeframes due to its innovative architecture which utilizes parallel computing techniques. Studies have shown that when tasked with complex data retrieval operations, the system can outperform traditional search methods by significant margins, often reducing latency and increasing throughput rates. This is attributed to DeepSeek's advanced indexing methodologies combined with machine learning algorithms for pattern recognition which expedite query resolution processes effectively. In various benchmark tests conducted across different industries ranging from financial services to academic research fields, the results consistently highlighted that deploying DeepSeek resulted in a notable reduction of time taken per search operation 